In [ ]:
import torch
from torch import nn
from gtda.homology import VietorisRipsPersistence as vrp
from gtda.homology import FlagserPersistence as flp
import plotly.express as px
import plotly.figure_factory as ff
import pandas as pd
from itertools import chain, combinations
import numpy as np
from tqdm import tqdm
from gdeep.utility.optimisation import PersistenceGradient
import warnings
%load_ext autoreload
%autoreload 2


# Differentiating persistent diagrams

## Notation

 - $K$ is the simplicial complex

 - $p$ is the number of finite bars (generated by *positive simplices* and eliminated by *negative simplices*)

 - $q$ is the number of infinitely persistent features.

This holds: $|K| = 2p + q$

## Description of the problem

Let us focus on VR, as the other filtrations are done similarly.

The first step is to notice that a filtration can be seen as a map 
$\Phi: A \to \mathbb R^{|K|} $. 

Given a point cloud with n points and d features, the space A can be seen as 
$A=(\mathbb R^d)^n$. 

The map $\Phi$ is given explicitly: given X a point cloud (i.e. an element of $(\mathbb R^d)^n$), 

$\Phi_{\sigma}(X)=max_{i,j \in \sigma}||x_i-x_j||.$ 

This is the $\sigma$ component of the whole $\Phi$.

Here sigma is a simplex, i.e. a coordinate of $\mathbb R^{|K|}$.

A persistent diagram requires:
1. The identification of $\mathbb R^{|K|} =(\mathbb R^2)^p \times \mathbb R^q$
2. The pairing of positive with negative simplices and the identification of unpaired positive simples.

Hence 

$Pers:Filt_K \subset \mathbb R^{|K|} \to (\mathbb R^2)^p \times \mathbb R^q, \Phi(X) \mapsto D = \cup_i^p (\Phi_{\sigma_{i_1}}(X) , \Phi_{\sigma_{i_2}}(X) ) \times \cup_j^q (\Phi_{\sigma_j}(X),+\infty).$

Of course, $|K|=2p + q$.

Finally, we can also define persistence functions: they are functions like $E: (\mathbb R^2)^p \times \mathbb R^q \to \mathbb R$, invariant under permutations of $p$ and $q$ .
e.g.
$E(D)=\sum_i^p|d_i-b_i|^2$

We can now define a loss function $L:= E.Pers.\Phi : A \to \mathbb R, A = (\mathbb R^d)^n$ as before.

Can we compute the gradient of $L$ with respect to the point cloud? Observe that Pers is merely a permutation of the coordinates, thus its partial derivatives w.r.t. the filtration are either 1 or 0.
Thus, since all the components of $L$ are differentiable, so is $L$ by Leibnitz rule.
One can implement it is Pytorch using `autograd`.

# Build the point cloud
We start with a random 2D point cloud.

In [ ]:
hom_dim = (0, 1, 2)
Xp = torch.rand((10, 4), requires_grad=False)
X_arr = Xp.detach().numpy().copy()
df = pd.DataFrame(Xp, columns=["x" + str(jj) for jj in range(len(Xp[0]))])

px.scatter(df, x="x0", y="x1")


# SGD over topology!

To gradient descend (for the moment, the algorithm is not stochastic) over the topological loss function $L=-\sum_i^p |\epsilon_{i2}-\epsilon_{i1}|+ \lambda \sum_{x \in X} ||x||_2^2$ it is enough to initialise the 
`PersistenceGradient` class and run the `SGD()` method. For clarity, $\epsilon_{i1}$ is the filtration value of the $i$-th *positive simplex*, while $\epsilon_{i2}$ of the $i$-th *negative simplex*. 

In [ ]:
pg = PersistenceGradient(
    homology_dimensions=hom_dim, zeta=0.1, max_edge_length=0.5, collapse_edges=False
)
fig, fig3d, loss_val = pg.sgd(Xp, 0.07, 32)

# plot of the evolution over the GD iterations
fig.show()
# plot of the evolution over the GD iterations in 3D
# fig3d.show()
# plot of the loss function
px.line(loss_val)


In [ ]:
df = pd.DataFrame(
    Xp.detach(), columns=["x" + str(jj) for jj in range(len(Xp.detach()[0]))]
)

px.scatter(df, x="x0", y="x1")


# Comparison between persistence diagrams
Plot the PD at the beginning and at the end of the optimisation

In [ ]:
# plot persistence diagram
vr = vrp(homology_dimensions=hom_dim)

try:
    vr.fit_transform_plot([X_arr])
except ValueError:
    warnings.warn("Most likely the array is empty...")
    pass


In [ ]:
try:
    vr.fit_transform_plot([Xp.detach()])
except ValueError:
    warnings.warn("Most likely the array is empty...")
    pass


# Application to weighterd graphs

The algorithm can be applied as is to weighted graphs as well.

Import the weighted graph as a square tensor, where the entry $(i,j)$ is the weight of the edge $i \to j$.

In [ ]:
dist = torch.rand((11, 11))  # simulate the weighted graph
dist_arr = dist.detach().numpy().copy()
pg = PersistenceGradient(
    homology_dimensions=hom_dim, zeta=0.0, collapse_edges=False, metric="precomputed"
)
fig, fig3d, loss_val = pg.sgd(dist, 0.1, 10)
# plot of the loss function
px.line(loss_val)


In [ ]:
# persistent homology before the optimisation
vr = vrp(metric="precomputed", homology_dimensions=hom_dim)
vr.fit_transform_plot([dist_arr])


In [ ]:
# persistent homology after the optimisation
try:
    vr.fit_transform_plot([dist.detach().numpy()])
except ValueError:
    warnings.warn("Most likely the array is empty...")
    pass


# Directed graphs
The algorithm also works for directed graphs!

In [ ]:
n = 11
dist = torch.rand((n, n)) + 1  # simulate the weighted directed graph
dist = dist * (torch.ones(n, n) - torch.eye(n, n))
dist_arr = dist.detach().numpy().copy()
pg = PersistenceGradient(
    homology_dimensions=hom_dim,
    zeta=0.01,
    collapse_edges=False,
    metric="precomputed",
    directed=True,
)
fig, fig3d, loss_val = pg.sgd(dist, 0.1, 10)
# plot of the loss function
px.line(loss_val)


In [ ]:
# persistent homology before the optimisation
fp = flp(homology_dimensions=hom_dim)
try:
    fp.fit_transform_plot([dist_arr])
except ValueError:
    warnings.warn("Most likely the array is empty...")
    pass


In [ ]:
# persistent homology after the optimisation
try:
    fp.fit_transform_plot([dist.detach().numpy()])
except ValueError:
    warnings.warn("Most likely the array is empty...")
    pass
